In [18]:
import os
import cv2
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVC
from keras_facenet import FaceNet
from mtcnn import MTCNN
import joblib

# Path to the dataset folder
dataset_folder = "data"

# Load the FaceNet model
model = FaceNet()

# Check if the SVM model is already saved
svm_model_path = "svm_model.pkl"
if os.path.exists(svm_model_path):
    # Load the SVM model
    classifier = joblib.load(svm_model_path)
    label_encoder = joblib.load("label_encoder.pkl")
else:
    # Initialize arrays to store face embeddings and labels
    face_embeddings = []
    labels = []

    # Iterate over subfolders in the dataset folder
    for subfolder_name in os.listdir(dataset_folder):
        subfolder_path = os.path.join(dataset_folder, subfolder_name)
        if not os.path.isdir(subfolder_path):
            continue

        # Load images from the subfolder
        for image_name in os.listdir(subfolder_path):
            image_path = os.path.join(subfolder_path, image_name)
            if not os.path.isfile(image_path):
                continue

            # Load and preprocess the image
            image = cv2.imread(image_path)
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            image = cv2.resize(image, (160, 160))  # Resize image to match FaceNet input size

            # Extract the face embedding
            face_embedding = model.embeddings(np.expand_dims(image, axis=0))[0]

            # Add the face embedding and corresponding label to the arrays
            face_embeddings.append(face_embedding)
            labels.append(subfolder_name)

    # Convert labels to numerical representation
    label_encoder = LabelEncoder()
    labels = label_encoder.fit_transform(labels)

    # Train a support vector machine (SVM) classifier
    classifier = SVC(kernel='linear')
    classifier.fit(face_embeddings, labels)

    # Save the SVM model and label encoder
    joblib.dump(classifier, svm_model_path)
    joblib.dump(label_encoder, "label_encoder.pkl")

# Test the trained model on a new image
test_image_path = "group2.jpg"
test_img = cv2.imread(test_image_path)
test_image = cv2.cvtColor(test_img, cv2.COLOR_BGR2RGB)
test_image_resized = cv2.resize(test_image, (160, 160))
test_image_resized = cv2.resize(test_img, (160, 160))# Resize image to match FaceNet input size

# Detect faces in the test image using MTCNN
detector = MTCNN()
faces = detector.detect_faces(test_image)

# Iterate over detected faces
for face in faces:
    # Extract the bounding box coordinates
    x, y, w, h = face['box']
    
    # Extract the face region
    face_image = test_image[y:y+h, x:x+w]
    face_image_resized = cv2.resize(face_image, (160, 160))  # Resize face region to match FaceNet input size

    # Extract the face embedding for the face region
    test_face_embedding = model.embeddings(np.expand_dims(face_image_resized, axis=0))[0]

    # Predict the label for the face region
    predicted_label = classifier.predict([test_face_embedding])

    # Get the confidence scores for all classes
    confidence_scores = classifier.decision_function([test_face_embedding])
    max_confidence_index = np.argmax(confidence_scores)
    max_confidence_score = confidence_scores[0][max_confidence_index]
    
    # Convert the predicted label back to the original class name
#     predicted_person = label_encoder.inverse_transform(predicted_label)[0]
    predicted_person = label_encoder.classes_[predicted_label[0]]

    # Determine the confidence threshold
    confidence_threshold = 2.3

    # Draw rectangle around the face
    if max_confidence_score >= confidence_threshold:
        text = f"{predicted_person} ({int((max_confidence_score*100)-100 )}% confidence)"
        color = (0, 255, 0)
    else:
        text = "Unknown"
        color = (0, 0, 255)

    cv2.rectangle(test_img, (x, y), (x + w, y + h), color, 2)
    cv2.putText(test_img, text, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 1)

# Display the test image
cv2.imshow("Test Image", test_img)
cv2.waitKey(0)
cv2.destroyAllWindows()


1/1 [==============================] - 0s 77ms/step


In [12]:
import os
import cv2
import numpy as np
from keras_facenet import FaceNet
from mtcnn import MTCNN
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

# Function to load images from a directory and assign labels to them
def load_images_from_folder(folder):
    images = []
    labels = []
    for subfolder in os.listdir(folder):
        subfolder_path = os.path.join(folder, subfolder)
        if os.path.isdir(subfolder_path):
            for filename in os.listdir(subfolder_path):
                img_path = os.path.join(subfolder_path, filename)
                image = cv2.imread(img_path)
                if image is not None:
                    images.append(image)
                    labels.append(subfolder)
    return images, labels

# Load the training dataset
train_folder = 'data/train/'
train_images, train_labels = load_images_from_folder(train_folder)

# Convert labels to unique integer values
label_to_id = {label: idx for idx, label in enumerate(set(train_labels))}
train_labels = np.array([label_to_id[label] for label in train_labels])

# Preprocess the training images
train_images = [cv2.resize(image, (160, 160)) for image in train_images]
train_images = np.array(train_images) / 255.0

# Initialize the CNN model
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(160, 160, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(256, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(len(label_to_id), activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Check if saved model exists
saved_model_path = 'model.h5'
if os.path.exists(saved_model_path):
    # Load the saved model
    model.load_weights(saved_model_path)
else:
    # Train the model
    model.fit(train_images, train_labels, epochs=10, batch_size=32)
    # Save the model
    model.save_weights(saved_model_path)

# Initialize MTCNN for face detection
detector = MTCNN()

# Function to perform face recognition on test images
def recognize_faces(test_image):
    # Detect faces in the test image
    faces = detector.detect_faces(test_image)

    for face in faces:
        # Extract the face region
        x, y, w, h = face['box']
        face_img = test_image[y:y+h, x:x+w]

        # Preprocess the face image
        face_img = cv2.resize(face_img, (160, 160))
        face_img = np.expand_dims(face_img, axis=0) / 255.0

        # Perform face recognition
        predicted_label = model.predict(face_img)[0]
        confidence = np.max(predicted_label)

        # Set a threshold for confidence
        threshold = 0.1

        if confidence > threshold:
            # Find the corresponding class label
            label_index = np.argmax(predicted_label)
            predicted_class = list(label_to_id.keys())[list(label_to_id.values()).index(label_index)]
#             text = predicted_class
            text = f"{predicted_class} ({int((confidence*100) )}% confidence)"
        else:
            text = 'Unknown'

        # Draw rectangle around the face and put text
        cv2.rectangle(test_image, (x, y), (x+w, y+h), (0, 255, 0), 2)
        cv2.putText(test_image, text, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

    # Display the test image
    cv2.imshow('Face Recognition', test_image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

# Load and recognize faces in test images
test_folder = cv2.imread("group3.jpg")
recognize_faces(test_folder)


Epoch 1/10
6/6 [==============================] - 8s 1s/step - loss: 2.3779 - accuracy: 0.0486
Epoch 2/10
6/6 [==============================] - 7s 1s/step - loss: 2.2419 - accuracy: 0.1243
Epoch 3/10
6/6 [==============================] - 9s 1s/step - loss: 2.1337 - accuracy: 0.2216
Epoch 4/10
6/6 [==============================] - 7s 1s/step - loss: 1.9142 - accuracy: 0.3405
Epoch 5/10
6/6 [==============================] - 7s 1s/step - loss: 1.6525 - accuracy: 0.4811
Epoch 6/10
6/6 [==============================] - 7s 1s/step - loss: 1.2565 - accuracy: 0.6000
Epoch 7/10
6/6 [==============================] - 7s 1s/step - loss: 1.0468 - accuracy: 0.6324
Epoch 8/10
6/6 [==============================] - 7s 1s/step - loss: 0.7171 - accuracy: 0.7784
Epoch 9/10
6/6 [==============================] - 7s 1s/step - loss: 0.5116 - accuracy: 0.8216
Epoch 10/10
1/1 [==============================] - 0s 32ms/step


In [15]:
test_folder = cv2.imread("ayaanlone.jpg")
recognize_faces(test_folder)

1/1 [==============================] - 0s 33ms/step
